In [10]:
!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd 
import json
import ast
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
df = pd.read_csv("cgpt_corr_resp_991.csv")
df['answer_embedding'] = df['corr_answer_sentence'].apply(lambda x: model.encode(x))
# df['explanation_embedding'] = df['explanation'].apply(lambda x: model.encode(x))
# df

In [ ]:
s1 = []
s2 = []
for i in range(df.shape[0]):
    try:
        s = df.iloc[i].explanation#.replace("Explanation:","")
        s = s.split(".")

        s1.append(s[0])
        s2.append(s[1])
    except:
        s1.append(df.iloc[i].explanation)
        s2.append("")
s1_ed = []
for i in s1:
    try:
        x = i.replace("Explanation:","")
    except:
        x = i
    # try:
    #     x = model.encode(x)
    # except:
    #     x = [0]*len(x.split(" "))
    s1_ed.append(x)
df['s1'] = s1_ed
df["s2"] = s2
def model_enc(x):
    try:
        return model.encode(x)
    except:
        return [0]
df['explanation_embedding_1'] = df['s1'].apply(lambda x: model_enc(x))
df['explanation_embedding_2'] = df['s2'].apply(lambda x: model_enc(x))
df
     

,id,correct_answer,chat_gptanswer,correct,explanation,context,context_extended,question,answers,corr_answer_sentence,answer_embedding,s1,s2,explanation_embedding_1,explanation_embedding_2
0,train_1,0,0,True,Explanation: The patient's argument attempts ...,Patient: Pharmacists maintain that doctors sho...,Patient: Pharmacists maintain that doctors sho...,The patient's argument proceeds by,['attempting to discredit a position by questi...,attempting to discredit a position by question...,"[-0.005082775, 0.072952196, 0.012256379, 0.005...",The patient's argument attempts to discredit...,,"[0.01682029, 0.09985238, -0.0377104, -0.071274...","[-0.11883836, 0.04829866, -0.0025481812, -0.01..."
1,train_2,1,1,True,The reasoning in this scenario involves a cha...,Paula will visit the dentist tomorrow morning ...,Paula will visit the dentist tomorrow morning ...,The pattern of reasoning displayed above most ...,"['If Marge goes to the bank today, Lauren will...",Kevin will wash his car tomorrow only if Britt...,"[0.029819887, 0.08411792, 0.14391474, -0.05186...",The reasoning in this scenario involves a cha...,"In this case, Paula's visit to the dentist is...","[-0.0029536928, 0.07214309, 0.052675884, 0.039...","[0.03975027, 0.0055491193, 0.020274699, 0.0013..."
2,train_3,1,1,True,Explanation: The argument's conclusion that s...,Some theorists argue that literary critics sho...,Some theorists argue that literary critics sho...,The argument's conclusion follows logically if...,"[""Any critic who is able to help readers make ...",If it is impossible to produce completely valu...,"[0.0043374756, 0.022740446, -0.10385152, -0.07...",The argument's conclusion that some theorist...,This is because the argument suggests that va...,"[0.020490583, 0.05457268, -0.08979924, -0.0524...","[0.05604011, 0.046158623, -0.060187213, -0.058..."
3,train_7,1,1,True,Explanation: The argument claims that peat ha...,Opponents of peat harvesting in this country a...,Opponents of peat harvesting in this country a...,"Which one of the following, if true, most stre...","[""The peat-harvesting industry of Ireland has ...",The original ecology of the peat-harvesting ar...,"[0.05405086, 0.049614366, 0.040399022, -0.0508...",The argument claims that peat harvesting wou...,Option 1 is irrelevant to the argument; optio...,"[0.002772455, 0.09749957, 0.026211351, -0.0493...","[-0.0029232732, 0.12467865, 0.022785624, -0.02..."
4,train_8,1,1,True,Explanation: The increase in the number of an...,Shortly after the power plant opened and began...,Shortly after the power plant opened and began...,"Each of the following, if true, could explain ...",['The discharged water was warmer than the nor...,"Because of the new plant, access to the part o...","[-0.024611307, -0.009641789, -0.010528517, -0....",The increase in the number of anglers fishin...,"The other options, such as warmer water, incr...","[0.023263931, 0.007174621, 0.0058969934, -0.01...","[0.046840735, 0.030975448, 0.1545019, 0.102228..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,train_1615,3,3,True,The flawed pattern of reasoning in the studen...,Student: If a person has an immunity to infect...,Student: If a person has an immunity to infect...,The student's argument is most similar in its ...,['Isabel said that she would take the medicati...,When business owners are subjected to excessiv...,"[0.08112812, -0.03586492, 0.118786454, -0.0501...",The flawed pattern of reasoning in the studen...,Both arguments commit the fallacy of oversimp...,"[0.031272512, 0.03772625, 0.08808754, -0.02315...","[0.027526112, 0.07469265, 0.048857823, 0.07580..."
987,train_1616,1,1,True,Explanation: Knowing how many people chose bo...,Columnist: George Orwell' s book 1984 has exer...,Columnist: George Orwell' s book 1984 has exer...,The answer to which one of the following quest...,['How many books had each person surveyed read...,How many people chose books other than 1984?,"[0.073527016, -0.0068512787, 0.023303181,

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
cos_sim_1 = []
cos_sim_2 = []
for i in range(df.shape[0]):
    try:
        sim_1 = util.pytorch_cos_sim(df.iloc[i].answer_embedding, df.iloc[i].explanation_embedding_1).numpy()
        cos_sim_1.append(sim_1[0][0])
    except:
        cos_sim_1.append(0)
    try:
        sim_2 = util.pytorch_cos_sim(df.iloc[i].answer_embedding, df.iloc[i].explanation_embedding_2).numpy()
        cos_sim_2.append(sim_2[0][0])
    except:
        cos_sim_2.append(0)
df['cos_sim_1'] = cos_sim_1
df['cos_sim_2'] = cos_sim_2
df

,id,correct_answer,chat_gptanswer,correct,explanation,context,context_extended,question,answers,corr_answer_sentence,answer_embedding,s1,s2,explanation_embedding_1,explanation_embedding_2,cos_sim_1,cos_sim_2
0,train_1,0,0,True,Explanation: The patient's argument attempts ...,Patient: Pharmacists maintain that doctors sho...,Patient: Pharmacists maintain that doctors sho...,The patient's argument proceeds by,['attempting to discredit a position by questi...,attempting to discredit a position by question...,"[-0.005082775, 0.072952196, 0.012256379, 0.005...",The patient's argument attempts to discredit...,,"[0.01682029, 0.09985238, -0.0377104, -0.071274...","[-0.11883836, 0.04829866, -0.0025481812, -0.01...",0.433918,0.105631
1,train_2,1,1,True,The reasoning in this scenario involves a cha...,Paula will visit the dentist tomorrow morning ...,Paula will visit the dentist tomorrow morning ...,The pattern of reasoning displayed above most ...,"['If Marge goes to the bank today, Lauren will...",Kevin will wash his car tomorrow only if Britt...,"[0.029819887, 0.08411792, 0.14391474, -0.05186...",The reasoning in this scenario involves a cha...,"In this case, Paula's visit to the dentist is...","[-0.0029536928, 0.07214309, 0.052675884, 0.039...","[0.03975027, 0.0055491193, 0.020274699, 0.0013...",0.406243,0.263828
2,train_3,1,1,True,Explanation: The argument's conclusion that s...,Some theorists argue that literary critics sho...,Some theorists argue that literary critics sho...,The argument's conclusion follows logically if...,"[""Any critic who is able to help readers make ...",If it is impossible to produce completely valu...,"[0.0043374756, 0.022740446, -0.10385152, -0.07...",The argument's conclusion that some theorist...,This is because the argument suggests that va...,"[0.020490583, 0.05457268, -0.08979924, -0.0524...","[0.05604011, 0.046158623, -0.060187213, -0.058...",0.901714,0.819149
3,train_7,1,1,True,Explanation: The argument claims that peat ha...,Opponents of peat harvesting in this country a...,Opponents of peat harvesting in this country a...,"Which one of the following, if true, most stre...","[""The peat-harvesting industry of Ireland has ...",The original ecology of the peat-harvesting ar...,"[0.05405086, 0.049614366, 0.040399022, -0.0508...",The argument claims that peat harvesting wou...,Option 1 is irrelevant to the argument; optio...,"[0.002772455, 0.09749957, 0.026211351, -0.0493...","[-0.0029232732, 0.12467865, 0.022785624, -0.02...",0.696203,0.455176
4,train_8,1,1,True,Explanation: The increase in the number of an...,Shortly after the power plant opened and began...,Shortly after the power plant opened and began...,"Each of the following, if true, could explain ...",['The discharged water was warmer than the nor...,"Because of the new plant, access to the part o...","[-0.024611307, -0.009641789, -0.010528517, -0....",The increase in the number of anglers fishin...,"The other options, such as warmer water, incr...","[0.023263931, 0.007174621, 0.0058969934, -0.01...","[0.046840735, 0.030975448, 0.1545019, 0.102228...",0.736461,0.344429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,train_1615,3,3,True,The flawed pattern of reasoning in the studen...,Student: If a person has an immunity to infect...,Student: If a person has an immunity to infect...,The student's argument is most similar in its ...,['Isabel said that she would take the medicati...,When business owners are subjected to excessiv...,"[0.08112812, -0.03586492, 0.118786454, -0.0501...",The flawed pattern of reasoning in the studen...,Both arguments commit the fallacy of oversimp...,"[0.031272512, 0.03772625, 0.08808754, -0.02315...","[0.027526112, 0.07469265, 0.048857823, 0.07580...",0.579602,0.238795
987,train_1616,1,1,True,Explanation: Knowing how many people chose bo...,Columnist: George Orwell' s book 1984 has exer...,Columnist: George Orwell' s book 1984 has exer...,The answer to which one of the following quest...,[

In [ ]:
# df.to_csv("data/sent_similarity_2.csv",index=False)